In [1]:
%matplotlib inline
# !pip install stargazer
# !pip install scipy
# !pip install plotly_express
# !pip install pandas-profiling
# !pip install linearmodels
# !pip install sklearn

import scipy
import numpy as np
import pandas as pd
import pandas.io.formats.style
import seaborn as sns
import statsmodels as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from IPython.display import HTML
import linearmodels as lm
from linearmodels import PanelOLS
import warnings
warnings.filterwarnings("ignore")
import math
# import sklearn

from linearmodels.panel import compare
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm_api


<a class="anchor" id="8. Difference in differences implementation"></a>


# cash flow is already level data   and there is no infor for No.of bank relations 

In [2]:
path="/Users/effie/Desktop/term3/research_module/data/"
data=pd.read_csv(path+"firm_clean/ready_for_regression.csv")

# path="/Users/mengxi/Desktop/financial research/final/"
# data=pd.read_csv(path+"211221_ready_for_regression.csv")

In [3]:
data

,Unnamed: 0,BvDID_firm,year,plz_firm,ort_,regionalkey_,bundesland,BvDID_bank_1,postcode_bank_1,data2025_1,...,bank_affected_degree,cf_ratio,gros_ratio,shfd_ratio,staf_ave,staf_ln,toas_lev,toas_ln,turn_lev,turn_ln
0,1,DE2010041272,2006.0,13407,"Berlin, Stadt",11,Berlin,DE13190,60311,265857.0,...,1,0.075024,NaN,0.042334,48871.675978,10.796953,81.248314,4.397510,128.884630,4.858918
1,2,DE2010041272,2007.0,13407,"Berlin, Stadt",11,Berlin,DE13190,60311,302841.0,...,1,0.091505,NaN,0.038756,49828.607527,10.816345,78.166190,4.358837,150.773620,5.015780
2,3,DE2010041272,2008.0,13407,"Berlin, Stadt",11,Berlin,DE13190,60311,359251.0,...,1,0.070409,NaN,0.031984,52628.015345,10.871004,84.965980,4.442251,199.344200,5.295033
3,4,DE2010041272,2009.0,13407,"Berlin, Stadt",11,Berlin,DE13190,60311,605765.0,...,1,0.061911,NaN,0.029387,44985.775956,10.714102,85.924355,4.453467,119.532216,4.783586
4,5,DE2010041272,2010.0,13407,"Berlin, Stadt",11,Berlin,DE13190,60311,623634.0,...,1,0.042470,NaN,0.027559,51404.482353,10.847481,89.440920,4.493578,146.464800,4.986785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7969,7970,DE8350106575,2011.0,84056,Landshut,9274,Bayern,DE13732,84028,3274.0,...,1,0.161112,NaN,0.202240,33063.769231,10.406193,1.546713,0.436132,2.317667,0.840561
7970,7971,DE8350106575,2012.0,84056,Landshut,9274,Bayern,DE13732,84028,3268.0,...,1,0.057209,NaN,0.216084,36853.846154,10.514715,1.542484,0.433394,2.320461,0.841766
7971,7972,DE8350164094,2006.0,85356,Freising,9178,Bayern,DE42769,85354,1159.0,...,1,0.230300,NaN,0.000000,17312.543860,9.759187,2.136564,0.759199,5.152020,1.639389
7972,7973,DE8350164094,2007.0,85356,Freising,9178,Bayern,DE42769,85354,1190.0,...,1,0.207050,NaN,0.076367,19426.391304,9.874388,2.013842,0.700044,6.110219,1.809963


In [4]:
data=data.drop(columns=['Unnamed: 0'])
data.rename(columns={"bank_affected":"firm_affected"},inplace=True)
data.rename(columns={"bank_affected_degree":"firm_affected_degree"},inplace=True)

data["firm_affected"].value_counts()


1    5596
0    2378
Name: firm_affected, dtype: int64

In [5]:
data["firm_affected_degree"].value_counts()

1    5345
0    2378
2     235
3      16
Name: firm_affected_degree, dtype: int64

In [6]:
data["cf_ratio"].value_counts()

0.088435    1
0.012991    1
0.144481    1
0.070066    1
0.113539    1
           ..
0.137680    1
0.117831    1
0.024563    1
0.046612    1
0.078836    1
Name: cf_ratio, Length: 7033, dtype: int64

In [7]:
data.columns

Index(['BvDID_firm', 'year', 'plz_firm', 'ort_', 'regionalkey_', 'bundesland',
       'BvDID_bank_1', 'postcode_bank_1', 'data2025_1', 'data2075_1',
       ...
       'firm_affected_degree', 'cf_ratio', 'gros_ratio', 'shfd_ratio',
       'staf_ave', 'staf_ln', 'toas_lev', 'toas_ln', 'turn_lev', 'turn_ln'],
      dtype='object', length=102)

<a class="anchor" id="8. Difference in differences implementation"></a>

---
# Difference in differences implementation
---

In [8]:
def did_est(variable,df):
    '''
    arguments:variable of interest, dataset
    return: 2x2 dif-in-dif table
    '''
    
    # affected Before and after
    affected_before = round(df.loc[(df['firm_affected']==1) & (df['year_post']!=1),variable].mean(),2)
    affected_after  = round(df.loc[(df['firm_affected']==1) & (df['year_post']==1),variable].mean(),2)

    # unaffected Before and after
    unaffected_before = round(df.loc[(df['firm_affected']!=1) & (df['year_post']!=1),variable].mean(),2)
    unaffected_after  = round(df.loc[(df['firm_affected']!=1) & (df['year_post']==1),variable].mean(),2)
    
    did= pd.DataFrame(data=np.array([[affected_after, affected_before,affected_after - affected_before],
    [unaffected_after, unaffected_before, unaffected_after - unaffected_before]]), index=['affected','unaffected'],columns=['after','before','$\Delta$'])
    
    return did

In [9]:
did_empl=did_est('empl',data)
did_empl

,after,before,$\Delta$
affected,4.53,4.57,-0.04
unaffected,4.29,4.31,-0.02


In [10]:
print("Dif-in-dif estimate for empl is", round(did_empl['$\Delta$'][0]-did_empl['$\Delta$'][1],2))

Dif-in-dif estimate for empl is -0.02


In [11]:
did_staf=did_est('staf_ln',data)
did_staf

,after,before,$\Delta$
affected,10.73,10.68,0.05
unaffected,10.69,10.67,0.02


In [12]:
print("Dif-in-dif estimate for staff expense is", round(did_staf['$\Delta$'][0]-did_staf['$\Delta$'][1],2))

Dif-in-dif estimate for staff expense is 0.03


<a class="anchor" id="9. Regression-adjusted models"></a>

---
#  Regression-adjusted models
---

 The DiD estimator can easily be implemented using regression. This is
a convenient way to obtain estimates and the corresponding standard errors in one step.

Consider the DiD linear regression model:  

$$employment_{i,t} = \beta_1 (Post_t \times affected_i) + \beta_2 X_{it} + \beta_3 firm_i +\beta_4 time_i + \varepsilon_{i,t}$$

In [13]:
model1 = smf.ols(formula = "empl ~ firm_affected*year_post", data = data).fit()

In [14]:
print("Dif-in-dif coefficient estimate for no of employee is ",round(model1.params[3],3))

Dif-in-dif coefficient estimate for no of employee is  -0.009


In [15]:
model2 = smf.ols(formula = "staf_ln ~ firm_affected*year_post", data = data).fit()

In [16]:
print("Dif-in-dif coefficient estimate for staff expense is ",round(model2.params[3],3))

Dif-in-dif coefficient estimate for staff expense is  0.031


  

###  DiD  only include  fixed effects 
$$employment_{i,t} = \beta_1 (post_t \times affected_i) +u_i+v_t+ \varepsilon_{i,t}$$

where $u_i$ isw a firm fixed effect and $v_t$ is a time fixed effect.



In [17]:
data["affected_after"] = data.firm_affected*data.year_post
df_fe = data.set_index(['BvDID_firm','year'])
df_fe

plz_firm           ort_  regionalkey_ bundesland  \
BvDID_firm   year                                                       
DE2010041272 2006.0     13407  Berlin, Stadt            11     Berlin   
             2007.0     13407  Berlin, Stadt            11     Berlin   
             2008.0     13407  Berlin, Stadt            11     Berlin   
             2009.0     13407  Berlin, Stadt            11     Berlin   
             2010.0     13407  Berlin, Stadt            11     Berlin   
...                       ...            ...           ...        ...   
DE8350106575 2011.0     84056       Landshut          9274     Bayern   
             2012.0     84056       Landshut          9274     Bayern   
DE8350164094 2006.0     85356       Freising          9178     Bayern   
             2007.0     85356       Freising          9178     Bayern   
             2012.0     85356       Freising          9178     Bayern   

                    BvDID_bank_1  postcode_bank_1  data2025_1  data2075_1  \
BvDID_firm   year                                                           
DE2010041272 2006.0      DE13190            60311    265857.0     79992.0   
             2007.0      DE13190            60311    302841.0    103003.0   
             2008.0      DE13190            60311    359251.0    126692.0   
             2009.0      DE13190            60311    605765.0    177120.0   
             2010.0      DE13190            60311    623634.0    370294.0   
...                          ...              ...         ...         ...   
DE8350106575 2011.0      DE13732            84028      3274.0       548.0   
             2012.0      DE13732            84028      3268.0       564.0   
DE8350164094 2006.0      DE42769            85354      1159.0       102.0   
             2007.0      DE42769            85354      1190.0       144.0   
             2012.0      DE42769            85354      1258.0       102.0   

                     data11750_1  data30680_1  ...  cf_ratio  gros_ratio  \
BvDID_firm   year                              ...                         
DE2010041272 2006.0     253930.0          NaN  ...  0.075024         NaN   
             2007.0     291005.0         6.90  ...  0.091505         NaN   
             2008.0     340248.0        10.10  ...  0.070409         NaN   
             2009.0     582393.0        10.50  ...  0.061911         NaN   
             2010.0     599549.0        11.90  ...  0.042470         NaN   
...                          ...          ...  ...       ...         ...   
DE8350106575 2011.0       2819.0          NaN  ...  0.161112         NaN   
             2012.0       2785.0        21.27  ...  0.057209         NaN   
DE8350164094 2006.0       1088.0          NaN  ...  0.230300         NaN   
             2007.0       1117.0          NaN  ...  0.207050         NaN   
             2012.0       1154.0        11.94  ...       NaN         NaN   

                    shfd_ratio      staf_ave    staf_ln   toas_lev   toas_ln  \
BvDID_firm   year                                                              
DE2010041272 2006.0   0.042334  48871.675978  10.796953  81.248314  4.397510   
             2007.0   0.038756  49828.607527  10.816345  78.166190  4.358837   
             2008.0   0.031984  52628.015345  10.871004  84.965980  4.442251   
             2009.0   0.029387  44985.775956  10.714102  85.924355  4.453467   
             2010.0   0.027559  51404.482353  10.847481  89.440920  4.493578   
...                        ...           ...        ...        ...       ...   
DE8350106575 2011.0   0.202240  33063.769231  10.406193   1.546713  0.436132   
             2012.0   0.216084  36853.846154  10.514715   1.542484  0.433394   
DE8350164094 2006.0   0.000000  17312.543860   9.759187   2.136564  0.759199   
             2007.0   0.076367  19426.391304   9.874388   2.013842  0.700044   
             2012.0   0.364251           NaN        NaN   2.628000  0.966223   

                       turn_lev   turn_

In [18]:
#for empl
mod = lm.PanelOLS.from_formula('empl ~ affected_after + EntityEffects +TimeEffects ',df_fe)
clfe_empl = mod.fit(cov_type = 'clustered')
# cluster_entity = True, cluster_time=True)
print(clfe_empl)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   empl   R-squared:                        0.0015
Estimator:                   PanelOLS   R-squared (Between):             -0.0061
No. Observations:                7974   R-squared (Within):              -0.0084
Date:                Sun, Dec 26 2021   R-squared (Overall):             -0.0062
Time:                        13:47:18   Log-likelihood                    615.59
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      9.7661
Entities:                        1267   P-value                           0.0018
Avg Obs:                       6.2936   Distribution:                  F(1,6698)
Min Obs:                       1.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             7.8907
                            

In [19]:
#for staff expense
mod = lm.PanelOLS.from_formula('staf_ln ~ affected_after + EntityEffects +TimeEffects ',df_fe)
clfe_staf = mod.fit(cov_type = 'clustered')
# cluster_entity = True, cluster_time=True)
print(clfe_staf)

                          PanelOLS Estimation Summary                           
Dep. Variable:                staf_ln   R-squared:                        0.0007
Estimator:                   PanelOLS   R-squared (Between):              0.0016
No. Observations:                7195   R-squared (Within):               0.0103
Date:                Sun, Dec 26 2021   R-squared (Overall):              0.0017
Time:                        13:47:18   Log-likelihood                    1943.2
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      4.4595
Entities:                        1202   P-value                           0.0347
Avg Obs:                       5.9859   Distribution:                  F(1,5984)
Min Obs:                       1.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             3.9622
                            

<a class="anchor" id="9. Regression-adjusted models"></a>

---
#  replication of table 3

---

### Adding controls---Table 3 column 1    empl

The regression formulation of DiD also allows to include additional control variables.  
In a DiD you can control for other covariates that may differ between the groups, and the main reason for doing this is to have a more robust identification and decreasing the variance of the regression coefficient estimates.  

The DiD linear regression model becomes:

$$employment_{i,t} = \beta_1 (Post_t \times affected_i) + \beta_2 X_{it} + \beta_3 firm_i +\beta_4 time_i + \varepsilon_{i,t}$$
 

In [20]:
# data=data.dropna(axis=0)
name=["shfd_ratio", "gros_ratio", "cf_ratio"]
np.any(~np.isfinite(data[name]),axis=0)

shfd_ratio    False
gros_ratio     True
cf_ratio       True
dtype: bool

In [21]:
model_empl = smf.ols(formula = "empl ~ firm_affected*year_post + firm_affected + year_post + toas + turn + shfd_ratio ", data = data).fit()
model_empl.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,-6.7495,0.153,-44.144,0.000,-7.049,-6.450
firm_affected,0.0822,0.042,1.951,0.051,-0.000,0.165
year_post,-0.0008,0.048,-0.017,0.986,-0.094,0.093
firm_affected:year_post,-0.0550,0.056,-0.974,0.330,-0.166,0.056
toas,-0.0143,0.012,-1.229,0.219,-0.037,0.008
turn,0.6694,0.012,55.529,0.000,0.646,0.693
shfd_ratio,0.8089,0.057,14.086,0.000,0.696,0.921


### Adding controls and fixed effect---Table 3 column 2   empl



In [22]:
mod = lm.PanelOLS.from_formula('empl ~ affected_after + EntityEffects +TimeEffects + toas +turn +shfd_ratio',df_fe)
didfe_empl = mod.fit(cov_type = 'clustered')
# cluster_entity = True, cluster_time=True)
print(didfe_empl)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   empl   R-squared:                        0.0971
Estimator:                   PanelOLS   R-squared (Between):              0.8938
No. Observations:                5823   R-squared (Within):               0.0974
Date:                Sun, Dec 26 2021   R-squared (Overall):              0.9076
Time:                        13:47:19   Log-likelihood                    1222.2
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      126.03
Entities:                        1121   P-value                           0.0000
Avg Obs:                       5.1945   Distribution:                  F(4,4690)
Min Obs:                       1.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             9.2919
                            

### Adding controls---Table 3 column 3    staff



In [23]:
model_staf = smf.ols(formula = "staf_ln ~ firm_affected*year_post + firm_affected + year_post + toas +turn + shfd_ratio+ gros_ratio+ cf_ratio", data = data).fit()
model_staf.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,8.8576,0.676,13.112,0.000,7.503,10.212
firm_affected,0.3560,0.130,2.734,0.008,0.095,0.617
year_post,0.2788,0.142,1.966,0.054,-0.005,0.563
firm_affected:year_post,-0.2133,0.149,-1.428,0.159,-0.513,0.086
toas,0.2541,0.095,2.665,0.010,0.063,0.445
turn,-0.1571,0.118,-1.336,0.187,-0.393,0.079
shfd_ratio,0.0616,0.235,0.262,0.795,-0.410,0.533
gros_ratio,-0.1198,0.166,-0.721,0.474,-0.453,0.213
cf_ratio,0.0356,0.320,0.111,0.912,-0.606,0.677


### Adding controls---Table 3 column 4    staff



In [24]:
mod = lm.PanelOLS.from_formula('staf_ln ~ affected_after + EntityEffects +TimeEffects + toas +turn + shfd_ratio',df_fe)
didfe_staf = mod.fit(cov_type = 'clustered')
# cluster_entity = True, cluster_time=True)
print(didfe_staf)

                          PanelOLS Estimation Summary                           
Dep. Variable:                staf_ln   R-squared:                        0.0568
Estimator:                   PanelOLS   R-squared (Between):              0.6421
No. Observations:                5777   R-squared (Within):               0.0809
Date:                Sun, Dec 26 2021   R-squared (Overall):              0.6480
Time:                        13:47:19   Log-likelihood                    1548.6
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      70.117
Entities:                        1104   P-value                           0.0000
Avg Obs:                       5.2328   Distribution:                  F(4,4661)
Min Obs:                       1.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             2.3878
                            

## compare col 2 and  4 

In [25]:
print(compare({"(1)Log_employee": didfe_empl, "(2)Log_average_wage": didfe_staf}, stars = True, precision = "std_errors"))

                      Model Comparison                     
                        (1)Log_employee (2)Log_average_wage
-----------------------------------------------------------
Dep. Variable                      empl             staf_ln
Estimator                      PanelOLS            PanelOLS
No. Observations                   5823                5777
Cov. Est.                     Clustered           Clustered
R-squared                        0.0971              0.0568
R-Squared (Within)               0.0974              0.0809
R-Squared (Between)              0.8938              0.6421
R-Squared (Overall)              0.9076              0.6480
F-statistic                      126.03              70.117
P-value (F-stat)                 0.0000              0.0000
=====================      ============         ===========
affected_after               -0.0482***            0.0289**
                               (0.0153)            (0.0139)
shfd_ratio                       0.0040 

## compare col 1 2 3 and 4  Mengxi's

In [26]:

exog_vars = ['affected_after','firm_affected','year_post','toas_ln','turn_ln','shfd_ratio','cf_ratio','firm_affected_degree']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.empl,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after','toas_ln','turn_ln','shfd_ratio','cf_ratio']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.empl,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after','firm_affected','year_post','toas_ln','turn_ln','shfd_ratio','cf_ratio','firm_affected_degree']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.staf_ln,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after','toas_ln','turn_ln','shfd_ratio','cf_ratio']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.staf_ln,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)Log_employee": res_fe_1, "(2)Log_employee": res_fe_2, "(3)Log_average_wage": res_fe_3, "(4)Log_average_wage": res_fe_4}, stars = True, precision = "std_errors"))

                                        Model Comparison                                        
                         (1)Log_employee (2)Log_employee (3)Log_average_wage (4)Log_average_wage
------------------------------------------------------------------------------------------------
Dep. Variable                       empl            empl             staf_ln             staf_ln
Estimator                       PanelOLS        PanelOLS            PanelOLS            PanelOLS
No. Observations                    5623            5623                5616                5616
Cov. Est.                      Clustered       Clustered           Clustered           Clustered
R-squared                         0.5302          0.1342              0.1359              0.0239
R-Squared (Within)               -0.1561          0.1373              0.0435              0.0524
R-Squared (Between)               0.5582          0.4224              0.1637              0.1112
R-Squared (Overall)           

## Heteroskedastic-Corrected and clustered standard errors 




In [27]:
# empl 
cluster_mod = smf.ols(formula='empl ~ affected_after', data=df_fe).fit(cov_type='nonrobust') 
fe_groups = df_fe.copy()
for i in ['empl', 'firm_affected',"year_post"]:
    fe_groups = fe_groups[pd.notnull(fe_groups[i])]# drop any entries with missing data from the model
result=cluster_mod.get_robustcov_results(cov_type='cluster',groups=fe_groups['sizecat'])#adjust SEs
result.params
print(f'Dif-in-dif coefficient for empl with standard errors clustered by sizecat: coeff is {result.params[1]:.3f} and std.err. is {result.bse[1]:.3f} --> pvalue {result.pvalues[1]:.3f}')

Dif-in-dif coefficient for empl with standard errors clustered by sizecat: coeff is 0.098 and std.err. is 0.052 --> pvalue 0.157


In [28]:
# staf
cluster_mod = smf.ols(formula='staf ~ affected_after', data=df_fe).fit(cov_type='nonrobust') 
fe_groups = df_fe.copy()
for i in ['staf', 'firm_affected',"year_post"]:
    fe_groups = fe_groups[pd.notnull(fe_groups[i])]# drop any entries with missing data from the model
result=cluster_mod.get_robustcov_results(cov_type='cluster',groups=fe_groups['sizecat'])#adjust SEs
result.params
print(f'Dif-in-dif coefficient for staf with standard errors clustered by sizecat: coeff is {result.params[1]:.3f} and std.err. is {result.bse[1]:.3f} --> pvalue {result.pvalues[1]:.3f}')

Dif-in-dif coefficient for staf with standard errors clustered by sizecat: coeff is 0.142 and std.err. is 0.054 --> pvalue 0.078
